### Обучаю Байесовскую сеть

In [1]:
#Импорты библиотек и вспомогательных функций
%run C:\Users\Илья\Desktop\Work\support_functions.ipynb

Automatically created module for IPython interactive environment


1. Переделать код, чтобы он принимал данные, автоматически их переводил в дискретный вид, обучал БС и выводил качество (табличку качества). Пробую обучить на данных без пропусков и с 1 пропуском.
2. Провести эксперимент с БС и описать, сохранить ноутбук.

#### Смотрим датасеты, отбираем фичи для моделирования

In [7]:
#Функция генерации иlack list если указаны targets
def black_list_generating(targets,all_features):
    non_targets=list(set(all_features)-set(targets))
    edges_for_black=list(itertools.product(targets,non_targets))
    black_list=pd.DataFrame(edges_for_black,columns=["from","to"])
    return black_list,edges_for_black

#Функция генерации white list если указаны targets
def fixed_edges_generating(targets,aprior_predictors):
    fixed_edges=list(itertools.product(targets,aprior_predictors))
    return fixed_edges

#Функция деления выборки на тренировочную и тестовую часть
def train_test(df):
    #Перемешиваем выборку
    df=df.sample(random_state=0,frac=1)
    
    #Вытаскиваем прогнозную величину
    target=targets[0]
    
    X,y=df.drop(target,axis=1),df[target]
    
    #Делим на тест и тренировку
    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                        test_size=0.33, random_state=0)
    
    df_train=pd.concat([X_train,y_train],axis=1)
    df_test=pd.concat([X_test,y_test],axis=1)
    
    return df_train,df_test


# #Создаем таблицу с метриками
# pd.DataFrame(columns=["target","disease",'methodtype',"scoretype",
#                       'methodtype_param', 'accuracy','precision','recall',
#                       'F1 score']).to_csv("Data/BN_metrics_inter_diagnosis_without_other.csv",index=None)


#Функция расчета метрик и их сохранения в единый файл, для набора признаков из targets
def metrics_calculation(model, df_test, methodtype, scoretype, methodtype_param):
    
    for target in targets:

        adjmat=model['adjmat']

        predictors=[k for k,v in adjmat.sum().to_dict().items() if (v!=0)&(k!=target)&(k not in targets)] #!

        #predicted=bn.predict(model, df_test[predictors], variables=[target])


        predicted = [np.argmax(bn.inference.fit(model, 
                                     variables=[target], 
                                     evidence=row[predictors].to_dict()).values) for i,row in tqdm_notebook(
            df_test.iterrows())]#! скорее всего не сможем предсказать без таргетов

        print(predicted)

        answers=df_test[target]

        accuracy=accuracy_score(answers, predicted)
        precision=precision_score(answers, predicted, average='weighted'),
        recall=recall_score(answers, predicted, average='weighted'),
        F1_score=f1_score(answers, predicted, average='weighted')
        print({f"accuracy {accuracy}"},precision,recall,F1_score)

        all_metrics=pd.read_csv("Data/BN_metrics_inter_diagnosis_without_other.csv")
        all_metrics=all_metrics.append(pd.DataFrame([[target,disease, methodtype, scoretype,
                                         methodtype_param, accuracy, precision[0], recall[0],
                                         F1_score]],
                            columns=all_metrics.columns.tolist()))
        print(all_metrics)
        all_metrics.to_csv("Data/BN_metrics_inter_diagnosis_without_other.csv",index=None)


        
        
        
#Функция построения БС
def create_Bayesian_network(df,methodtype,methodtype_param,scoretype,disease,targets=None,white_list=None,black_list=None,ind_graph=0):

                             
    #Если есть зависимые и целевые переменные, скрипт запрещает ребра от целевых к зависимым
    if (targets!=None)&(methodtype=="hc"):
        black_list,edges_for_black=black_list_generating(targets=targets,all_features=df.columns.tolist())
        bw_list_method="edges" #
        
        #Генерируем список заданных ребер
#         aprior_predictors=["Состояние_пациента","С-реактивный белок (СРБ) колич."]
#         fixed_edges=fixed_edges_generating(targets,aprior_predictors)
        
    else:
        bw_list_method=None
        fixed_edges=None
    
   
    
#     #Перемешиваем выборку, и делим на тест и тренировку
#     df_train,df_test=train_test(df)
    print(df.shape)
    DAG=bn.structure_learning.fit(df=df,#,
                                  methodtype=methodtype,
                                  scoretype=scoretype,
                                  bw_list_method=bw_list_method,
                                  black_list=black_list,
                                  white_list=white_list,
#                                   fixed_edges=fixed_edges
                                 )
    
    
    if isinstance(white_list,list)|isinstance(black_list,list):
        second_folder="with_black_white_edges"
    else:
        second_folder="without_black_white_edges"

    DAG['adjmat'].astype(int).to_csv(f"Data/adjacency matrices/{second_folder}/{ind_graph}_{disease}_{methodtype}_{scoretype}.csv")

#     bn.plot(DAG, interactive=True, params_interactive = {'height':'800px', 'width':'70%', 'layout':None, 'bgcolor':'#0f0f0f0f'})

    print(f"Структура БС с параметрами {methodtype},{scoretype},{methodtype_param},{disease}, найдена успешно")
    
#     #Обучаем и валидируем модель
#     model=bn.parameter_learning.fit(DAG,df=df_train,methodtype=methodtype_param)                        
#     metrics_calculation(model, df_test, methodtype, scoretype, methodtype_param)
    
#     print(f"Обучение и валидация БС с параметрами {methodtype},{scoretype},{disease},{methodtype_param} произведена успешно")
    


#Функция построения и сохранения всех возможных вариантов 
def create_all_types_BN(df,disease,targets):
    methodtypes=["chow-liu","hillclimbsearch","constraintsearch"] #"hillclimbsearch","constraintsearch","exhaustivesearch""chow-liu",,',"constraintsearch","chow-liu","hc"
    scoretypes=["bic", "bdeu","k2"] #
    
    methodtype_params=['bayes'] #,"" ,"maximumlikelihood"
    white_list=[None]
    black_list=[None]
    
    #Вводим индекс для понимания какой граф брать
    ind_graph=0
    for method,methodtype_param,score,white_list,black_list in tqdm_notebook(list(itertools.product(methodtypes,
                                                                     methodtype_params,
                                                                     scoretypes,
                                                                     white_list,
                                                                     black_list))):

        create_Bayesian_network(df,method,methodtype_param,score,disease,targets,white_list,black_list,ind_graph)
        ind_graph+=1


In [8]:
def generating_dataset():
    df_BN=pd.read_csv("Data/adjacency matrices/patienst_intervents_all_diagnosis_without_other_nodes_table_itrerv_without_groups.csv",
                      index_col="case_ID")
    return df_BN

#### Запускаем построение всех вариантов для отобранных данных без черных и белых листов

In [9]:
def experiment(df):
    create_all_types_BN(df=df,disease=disease,targets=None)

In [11]:
#Генерируем данные без пропусков для обучения
df_BN=generating_dataset()
disease="intervs_without_group_diagnosis_without_other"
# df_BN
#Запускаем эксперимент
experiment(df_BN)

  0%|          | 0/6 [00:00<?, ?it/s]

(1599, 31)
[bnlearn] >Computing best DAG using [hillclimbsearch]
[bnlearn] >Set scoring type at [bic]
Структура БС с параметрами hillclimbsearch,bic,bayes,intervs_without_group_diagnosis_without_other, найдена успешно
(1599, 31)
[bnlearn] >Computing best DAG using [hillclimbsearch]
[bnlearn] >Set scoring type at [bdeu]
Структура БС с параметрами hillclimbsearch,bdeu,bayes,intervs_without_group_diagnosis_without_other, найдена успешно
(1599, 31)
[bnlearn] >Computing best DAG using [hillclimbsearch]
[bnlearn] >Set scoring type at [k2]
Структура БС с параметрами hillclimbsearch,k2,bayes,intervs_without_group_diagnosis_without_other, найдена успешно
(1599, 31)
[bnlearn] >Computing best DAG using [constraintsearch]


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Структура БС с параметрами constraintsearch,bic,bayes,intervs_without_group_diagnosis_without_other, найдена успешно
(1599, 31)
[bnlearn] >Computing best DAG using [constraintsearch]


...

  0%|          | 0/5 [00:00<?, ?it/s]

Структура БС с параметрами constraintsearch,bdeu,bayes,intervs_without_group_diagnosis_without_other, найдена успешно
(1599, 31)
[bnlearn] >Computing best DAG using [constraintsearch]


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Структура БС с параметрами constraintsearch,k2,bayes,intervs_without_group_diagnosis_without_other, найдена успешно


Обучение и валидация БС с параметрами 
Progress_BSI18_Depression - пока лучший score 0.61 F1


Строим БСы для 1 таргета, и смотрим на scor, например для GAF, и так пробуем пока не получим хороший score
WQO_quality_of_life пробуем
Пробуем для Progress_ICF13_mean